In [1]:
import os
import xml.etree.ElementTree as ET

In [3]:
path="export_doc5445_cccc_178_alto_202505131407"

In [2]:
def count_textline_elements(alto_xml_path):
    # Parse the XML file
    tree = ET.parse(alto_xml_path)
    root = tree.getroot()
    
    # Define the namespace
    namespace = {'alto': 'http://www.loc.gov/standards/alto/ns-v4#'}
    
    # Find all TextLine elements
    textline_elements = root.findall('.//alto:TextLine', namespace)
    
    # Return the count of TextLine elements
    return len(textline_elements)

In [4]:
# No good, as this doesn't guarantee the correct order. We need to use the METS file instead
def get_xml_files(folder_path):
    files = os.listdir(folder_path)
    
    xml_files = [file for file in files if file.endswith('.xml')]

    return xml_files

In [14]:
def get_files_from_mets(mets_xml_path):
    tree = ET.parse(mets_xml_path)
    root = tree.getroot()

    # Define the namespace
    ns_mets = {'mets': 'http://www.loc.gov/METS/'}

    # Find all FLocat elements
    flocat_elements = root.findall('.//mets:FLocat', ns_mets)
    
    # Iterate over FLocat elements and collect their attributes
    # Could be much more pythony!
    href_list = []

    for flocat in flocat_elements:
        href_list.append(flocat.get('{http://www.w3.org/1999/xlink}href'))

    return href_list

In [16]:
countlist = []
for f in get_files_from_mets(os.path.join(path, "METS.xml")):
    countlist.append(count_textline_elements(os.path.join(path, f)))

In [17]:
print(countlist)

[5, 7, 11, 8, 3, 3, 6, 5, 3, 11, 6, 6, 7, 12, 11, 7, 15, 5, 5, 3, 20, 6, 11, 17, 14, 13, 8, 7, 16, 7, 7, 10, 14, 12, 24, 7, 10, 8, 13, 11, 10, 14, 5, 9, 1, 7, 6, 5, 9, 10, 4, 5, 10, 8, 6, 7, 13, 7, 5, 11, 12, 11, 8, 14, 13, 10, 16, 20, 18, 10, 4, 10, 9, 4, 12, 7, 12, 15, 20, 22, 20, 4, 13, 10, 7, 17, 26, 16, 16, 17, 25, 17, 11, 12, 17, 13, 14, 11, 14, 8, 17, 11, 7, 9, 6, 18, 13, 17, 19, 17, 15, 15, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 12, 11, 12, 9, 11, 7, 6, 15, 4, 7, 7, 20, 13, 13, 7, 13, 10, 14, 11, 25, 19, 20, 15, 14, 15, 17, 16, 19, 7, 13, 12, 7, 15, 18, 16, 19, 16, 11, 4, 7, 4, 8, 2, 7, 10, 5, 12, 6, 7, 10, 8, 3, 9, 6, 14, 2, 9, 6, 5, 6, 8, 3, 6, 8, 15, 10, 14, 8, 14, 11, 9, 2, 3, 6, 10, 8, 6, 3, 4, 0, 2, 0, 1, 5, 45, 7, 1, 0, 1, 40, 35, 24, 44, 12, 13, 31, 1, 1, 32, 41, 27, 5, 23, 63, 2